<a href="https://colab.research.google.com/github/sait1801/maximus_erectus/blob/main/powea_demo_gui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import the Necessary Libraries

In [88]:
import pandas as pd
import numpy as np
from datetime import date
import datetime

# Getting Dataframe

In [89]:
df_plot = pd.read_csv('/content/df_plot_full.csv')
# df_plot['Date'] = pd.to_datetime(df_plot['Date'], format='%d.%m.%Y') + pd.to_timedelta(df_plot['Hour'], unit='h')
df_plot['Date'] = pd.to_datetime(df_plot['Date'], format='%d.%m.%Y')


In [90]:
# df_plot.head(20)

In [91]:
##Day left finder
def day_left(current_charge,bat_cap,consumption):
  # print(f"current_charge = {current_charge},bat_cap = {bat_cap},consumption = {consumption}")
  kwh_left = (bat_cap * (current_charge-20)/100)
  # print(f"kwh_left for 20 to 80 in charge : {kwh_left}")
  # print(f"kwh required to charge  : {41.6-kwh_left}")

  return int(kwh_left/consumption)

import datetime

def days_between(date1, date2):
  date1 = datetime.datetime.strptime(date1, '%Y-%m-%d')
  date2 = datetime.datetime.strptime(date2, '%Y-%m-%d')
  return (date2 - date1).days



## EV BUS scheduler
def find_max_ren_tot_rate_bus(df, next_days,daily_consumption,date_index,increment):
    kw_renewed_bus =  []
    kw_charged_bus = []
    renewability_rates_bus = []

    for day_number in range(0,next_days):
      start_index = date_index + 24*day_number
      end_index = date_index+increment + 24*day_number
      renewability_rate =  df.loc[start_index : end_index]['ren/tot'].values.flatten()

      renewability_rates_bus.append(max(renewability_rate))
      kw_charged_bus.append(daily_consumption * (day_number+1))

      kw_renewed_bus.append(daily_consumption * (day_number+1) *renewability_rates_bus[-1] /100)


    max_value = max(renewability_rates_bus) #This is for the best for kwh renewed

    max_index = renewability_rates_bus.index(max_value) #This is for the best for kwh renewed

    return kw_renewed_bus[max_index],max_index, kw_charged_bus[max_index]

##EV car scheduler
def find_max_ren_tot_rate(df, next_days,daily_consumption,date_index,increment):
    kw_renewed =  []
    kw_charged = []
    renewability_rates = []
    day_inc = df_plot.loc[(df_plot['Date'] == pd.Timestamp(starting_date))].shape[0]

    for day_number in range(0,next_days):

      start_index = date_index + 24*day_number
      end_index = date_index+increment + 24*day_number

      renewability_rate =  df.loc[start_index : end_index]['ren/tot'].values.flatten()

      renewability_rates.append(max(renewability_rate))

      kw_renewed.append(daily_consumption * (day_number+1) *renewability_rates[-1] /100)
      kw_charged.append(daily_consumption * (day_number+1))

    print(f"ren rates of days: {renewability_rates}")
    print(f"kwh renewed rates of days: {kw_renewed}")


    max_value_ren_based =max(renewability_rates) #This is for the best for renewability rate
    max_index = renewability_rates.index(max_value_ren_based) #This is for the best for renewability rate
    print(f"max_index: {max_index}")


    try:
      if (max_index ==0 ):
        print("Index = 0")
        copy_renewability_rates = renewability_rates
        renewability_rates.sort()
        max_index = copy_renewability_rates.index(renewability_rates[-2])

        # ##HERE
        # if max_index==0:
        #   max_index +=1

    except:
      print(f"max index == 0 and the day left is 1 ")
      max_index +=1
      kw_renewed.append(kw_renewed[0])
      kw_charged.append(kw_charged[0])

    return kw_renewed[max_index],max_index, kw_charged[max_index]

#Date and Car Make Selector GUI

#Note ! : if your hour selection is 23-07 you must take +1 of each hour like 00-08

In [92]:
#@title Enter the Following Information
starting_date = '2023-03-09' #@param {type : "date"}
starting_hour = 20 #@param {type : "slider", min:0, max : 23, step:1}
car_battery_capacity = 100 #@param {type : "slider", min:42, max : 100, step:1}
end_date = '2023-03-10' #@param {type : "date"}
end_hour = 8 #@param {type : "slider", min:0, max : 23, step:1}

car_vs_bus = 'EV' #@param ['Bus', 'EV'] {type: "string"}

daily_average_electricity_consumption = 12 #@param {type : "slider", min: 1, max: 20}

if car_vs_bus == 'Bus':
  daily_average_electricity_consumption = 150
  car_battery_capacity = 450

print(f"daily_average_electricity_consumption : {daily_average_electricity_consumption}")

daily_average_electricity_consumption : 12


# Main Code

In [93]:
# df_plot.loc[(df_plot['Date'] == pd.Timestamp(starting_date))]

In [94]:
first_index= df_plot.loc[(df_plot['Date'] == pd.Timestamp(starting_date))]
first_index = first_index.loc[(first_index['Hour'] == starting_hour)]
first_index = first_index.index[0]

second_index= df_plot.loc[(df_plot['Date'] == pd.Timestamp(end_date))]
second_index = second_index.loc[(second_index['Hour'] == end_hour)]
second_index = second_index.index[0]

hour_difference  = second_index -first_index
hour_difference

12

In [95]:
if (starting_hour > end_hour): #23 -07
  increment = 24-starting_hour + end_hour
  df_plot = df_plot[df_plot['Hour'].between(starting_hour,24) | df_plot['Hour'].between(0,end_hour)]
elif (end_hour > starting_hour):
  increment = end_hour- starting_hour # #07-01
  df_plot = df_plot[df_plot['Hour'].between(starting_hour,end_hour)]

else:
  increment = 24
print(f"incremenet : {increment}")

incremenet : 12


In [96]:
# print(type(df_plot['Date'].loc[0]))
# pd.Timestamp(starting_date)

In [97]:
# df_plot.loc[(df_plot['Date'] == pd.Timestamp(starting_date))].head(5)

In [98]:
# date_index= df_plot.loc[(df_plot['Date'] == pd.Timestamp(starting_date))]
date_index= df_plot.loc[(df_plot['Date'] == pd.Timestamp(starting_date))]

date_index = date_index.loc[(date_index['Hour'] == starting_hour)]
date_index.index[0]

start_index = first_index
end_index = second_index
# print(start_index)
# print(end_index)
# print(end_index-start_index)

In [99]:
# print(end_index)

In [100]:
# df_plot.loc[start_index:end_index].head(50)

In [101]:
if (car_vs_bus == 'EV'):
  charged_percs = []
  starting_dates = []
  ren_rates= []
  charge_day = starting_date
  max_index = start_index
  start_index_clone = start_index
  current_charge=80

  try:
    while(True):
      day_to_next_charge = day_left(current_charge, car_battery_capacity,daily_average_electricity_consumption)
      day_between_dates = days_between(starting_date,end_date)
      print(f"day_between_dates: {day_between_dates}")
      print(f"day_to_next_charge : {day_to_next_charge}")

      if day_between_dates < day_to_next_charge:
        day_to_next_charge = day_between_dates

      try:
        max_kw_renewed, max_index,kw_consumed = find_max_ren_tot_rate(df_plot, day_to_next_charge,daily_average_electricity_consumption,start_index,increment)
      except Exception as e:
        print(f"An exception occurred: {str(e)}")
        print("Bitti")
        break
      print(f"max index ,kw_consumed, max renewed: {max_index, kw_consumed, max_kw_renewed}")

      # if max_index == 0:
      #   max_index +=1 ##HERE

      start_index += max_index*24
      print(f"start_index = {start_index}")




      ren_rates.append(max_kw_renewed)
      if start_index > end_index:
        print(f"EXITED SUCCESSFULLY :{start_index}, {end_index}")
        break

      current_charge =80
      print("============================")
  except:
    print(f"final cahrge day: {starting_date}")

else:
  print("BUS SELECTED")
  charged_percs = []
  starting_dates = []
  ren_rates= []
  charge_day = starting_date
  max_index = start_index
  start_index_clone = start_index
  try:
    while(True):
      day_to_next_charge = day_left(80, car_battery_capacity,daily_average_electricity_consumption)

      try:
        max_kw_renewed, max_index, kw_consumed = find_max_ren_tot_rate_bus(df_plot, day_to_next_charge,daily_average_electricity_consumption,start_index,increment)
      except Exception as e:
        print(f"An exception occurred: {str(e)}")
        print("Bitti")
        break


      print(f"max index ,kw_consumed: {max_index, kw_consumed, max_kw_renewed}")


      start_index += 24 ##??  increment



      ren_rates.append(max_kw_renewed)
      if start_index > end_index:
        print(f"EXITED SUCCESSFULLY :startin : {start_index}, end in {end_index}")
        break

      current_charge =100
      print("============================")
  except:
    print(f"final cahrge day: {starting_date}")


day_between_dates: 1
day_to_next_charge : 5
ren rates of days: [47.6320627510722]
kwh renewed rates of days: [5.715847530128664]
max_index: 0
Index = 0
max index == 0 and the day left is 1 
max index ,kw_consumed, max renewed: (1, 12, 5.715847530128664)
start_index = 19172
EXITED SUCCESSFULLY :19172, 19160


# Finalizing

In [102]:
ren_rates

[5.715847530128664]

In [103]:
# df_normal_bus = df_plot.loc[start_index_clone:end_index-12]
df_normal_bus = df_plot.loc[start_index_clone:end_index]

df_normal_bus = df_normal_bus[df_plot['Hour'] == starting_hour]
df_normal_bus.head(20)

<ipython-input-103-b8e26df5c2f1>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_normal_bus = df_normal_bus[df_plot['Hour'] == starting_hour]


,Date,Hour,Total,Flow,Dam,Wind,Solar,Jeo,ren/tot
19148,2023-03-09,20,39.34335,2.11865,5.21152,7.1927,0.0,1.18609,39.927866


In [104]:
# df_normal_bus = df_plot[df_plot['Hour'] == starting_hour]
# df_normal_bus = df_normal_bus.loc[start_index_clone:end_index]
# print(df_normal_bus.shape)
df_normal_consumed_bus = df_normal_bus['ren/tot'] * daily_average_electricity_consumption/100
df_normal_consumed_bus

19148    4.791344
Name: ren/tot, dtype: float64

In [105]:
# print(len(ren_rates))
# np.sum(ren_rates) #/(len(ren_rates)/df_normal_bus.shape[0])


# Results

In [106]:
print(f" renewable energy when NORMALLY charged : {np.sum(df_normal_consumed_bus)}")
print(f" renewable energy when SMARTLY charged : {np.sum(ren_rates)}")
print(f" renewable energy ratio NORMALLY charged : {np.sum(df_normal_consumed_bus)/(daily_average_electricity_consumption* len(df_normal_consumed_bus))}")
print(f" renewable energy ratio SMARTLY charged : {np.sum(ren_rates)/(daily_average_electricity_consumption* len(df_normal_consumed_bus))}")


 renewable energy when NORMALLY charged : 4.791343899286666
 renewable energy when SMARTLY charged : 5.715847530128664
 renewable energy ratio NORMALLY charged : 0.3992786582738888
 renewable energy ratio SMARTLY charged : 0.47632062751072196


In [107]:
(np.sum(ren_rates) - np.sum(df_normal_consumed_bus))/2

0.4622518154209989

## $$ Based Result

In [108]:
(np.sum(ren_rates) - np.sum(df_normal_consumed_bus))/df_normal_consumed_bus.shape[0]  *360 * 0.8 * 6 /1000


1.5975422740949725